In [ ]:
# Load the train data
#train = pd.read_csv('/train.csv')

In [ ]:
# Encoding the labels

encoded_labels = []
for i in range(len(train)):
    if train["label"][i] == "blues":
        encoded_labels.append(0)
    if train["label"][i] == "classical":
        encoded_labels.append(1)
    if train["label"][i] == "country":
        encoded_labels.append(2)
    if train["label"][i] == "disco":
        encoded_labels.append(3)
    if train["label"][i] == "hiphop":
        encoded_labels.append(4)
    if train["label"][i] == "jazz":
        encoded_labels.append(5)
    if train["label"][i] == "metal":
        encoded_labels.append(6)
    if train["label"][i] == "pop":
        encoded_labels.append(7)
    if train["label"][i] == "reggae":
        encoded_labels.append(8)
    if train["label"][i] == "rock":
        encoded_labels.append(9)


train["encoded_label"] = encoded_labels

# Extracting all the faetures from the dataset
features = train.drop(["filename", "label", "length", "encoded_label"], axis = "columns")
print(train)

In [ ]:
# Normalizing the features

features_norm = pd.DataFrame()
for column in features.columns:
    features_norm[column] = (features[column] - features[column].mean()) / features[column].std()

In [ ]:
# Finding correlation between features

import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (40, 40))
sns.heatmap(features_norm.corr(), ax = ax, annot = True, linewidths = .5)

In [ ]:
# Finding the distribution of different genres

genre_counts = train['label'].value_counts()
print(genre_counts)
plt.figure(figsize=(12, 6))
sns.barplot(x=genre_counts.index, y=genre_counts.values)
plt.title("Genre Distribution")
plt.xticks(rotation=90)
plt.xlabel("Genre")
plt.ylabel("Count")
plt.show()

In [ ]:
# Performing PCA to remove the correlated features and finding the most important features

from sklearn.decomposition import PCA

pca = PCA(n_components = 57)
features_final = pca.fit(features_norm)
n_pcs = pca.components_.shape[0]
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = list(features.columns)
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

df = pd.DataFrame(dic.items())


final_features = []
for i in range(len(df)):
    if df[1][i] not in final_features:
        final_features.append(df[1][i])
final_features = final_features[:]
train_final_pca = features_norm[final_features] # Removing correlated columns using PCA
train_final_all = features_norm # Keeping the correlated columns
train_final_no_mfcc2_mean = features_norm.drop(["mfcc2_mean"], axis = "columns") # Dropping only mfcc2_mean

In [ ]:
# Initializing and fitting the KNN models

import math
from sklearn.neighbors import KNeighborsClassifier

knn1 = KNeighborsClassifier(n_neighbors = 1,weights='distance',leaf_size= 50,algorithm='auto',p=1).fit(train_final_pca, train['encoded_label'])
knn2 = KNeighborsClassifier(n_neighbors = 1,weights='distance',leaf_size= 50,algorithm='auto',p=1).fit(train_final_no_mfcc2_mean, train['encoded_label'])
knn3 = KNeighborsClassifier(n_neighbors = 1,weights='distance',leaf_size= 50,algorithm='auto',p=1).fit(train_final_all, train['encoded_label'])

In [ ]:
# Reading the test dataset

test = pd.read_csv('/kaggle/input/pesurrcampusmicompetitionc/test.csv')
i_d = test['id']


# Extracting features from the test dataset
test_features = test.drop(["length", "id"], axis = "columns")

# Normalizing the features
test_features_norm = pd.DataFrame()
for column in test_features.columns:
    test_features_norm[column] = (test_features[column] - test_features[column].mean()) / test_features[column].std()


test_final_pca = test_features_norm[final_features] # Removing correlated columns using PCA
test_final_all = test_features_norm # Keeping the correlated columns (gives higher accuracy)
test_final_no_mfcc2_mean = test_features_norm.drop(["mfcc2_mean"], axis = "columns") # Dropping only mfcc2_mean

# Making predictions using the fitted KNN models
predictions1 = knn1.predict(test_final_pca)
predictions2 = knn2.predict(test_final_no_mfcc2_mean)
predictions3 = knn3.predict(test_final_all)

# Making a dataframe for the predictions and converting the dataframe into a csv file
submission_df = pd.DataFrame({'id': i_d ,'label': predictions3})
submission_df.to_csv('submission.csv', index=False)